# CMSC320 Final Tutorial - Do NBA Award Winners Deserve Their Awards?

In most professional sports leagues, a variety of awards are typically given to a player or players following the conclusion of the season to recognize them for their outstanding play.
These awards typically include recognition for being a league's Most Valuable Player, as well as recognition for offensive and defensive performances.

These players are typically from the best teams, and there is often controversy and discussion around whether or not a particular player was truly deserving of their award.

This final tutorial aims to answer three questions:

1. Given statistics from previous award winners, can we accurately predict the same award winner of any given season?
2. Given a set of statistics, can we predict if a hypothetical player would have won a particular in any given season?
3. Are award winners typically statistical outliers based on individual statistics, or does the success of their team play a significant role in award worthiness?

Given constraints on data availability from my sources, I will be utilizing data up to the 2016-17 NBA season, and data from future seasons to present will be used as testing to validate or invalidate my guiding questions.

## Future Plan

1. Contextualize MVP seasons
    1. Standardize variables by season, show how much of an outlier MVPs are in relevant statistics
2. Understand what statistics are most likely to contribute to MVP
    1. Does team W/L play a significant role or can sheer statistical greatness overcome poor team play?
3. Build predictor/percentage likelihood for MVP awards given a set of statistics
    1. Python functions


# Code:

In [15]:
import pandas as pd
import numpy as np
import sklearn as sk
from bs4 import BeautifulSoup
from urllib.request import urlopen
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)

player_data_df = pd.read_csv('datafiles/player_data.csv')
players_df = pd.read_csv('datafiles/Players.csv')
seasons_df = pd.read_csv('datafiles/Seasons_Stats.csv')

seasons_df = seasons_df.drop('Unnamed: 0', axis=1)
players_df = players_df.rename(columns={'collage': 'college'})

In [16]:
# Removing '*' from all players that had an all star season, and then marking their All-Star season in a separate column 
seasons_df['All Star Season'] = np.zeros(len(seasons_df), dtype=int)
for index, row in seasons_df.iterrows():
    
    if isinstance(row['Player'], str) == False:
        continue 
    if '*' in row['Player']:
        seasons_df.loc[index, 'Player'] = row['Player'][:len(row['Player'])-1]
        seasons_df.loc[index, 'All Star Season'] = 1
seasons_df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,blanl,OWS,DWS,WS,WS/48,blank2,OBPM,DBPM,BPM,VORP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,All Star Season
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,NaN,0.467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,3.6,3.5,NaN,NaN,NaN,NaN,NaN,NaN,144.0,516.0,0.279,NaN,NaN,NaN,144.0,516.0,0.279,0.279,170.0,241.0,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0,0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,NaN,0.387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,0.6,2.2,NaN,NaN,NaN,NaN,NaN,NaN,102.0,274.0,0.372,NaN,NaN,NaN,102.0,274.0,0.372,0.372,75.0,106.0,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0,0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,NaN,0.259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,2.8,3.6,NaN,NaN,NaN,NaN,NaN,NaN,174.0,499.0,0.349,NaN,NaN,NaN,174.0,499.0,0.349,0.349,90.0,129.0,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0,0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,NaN,0.395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN,NaN,22.0,86.0,0.256,NaN,NaN,NaN,22.0,86.0,0.256,0.256,19.0,34.0,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0,0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,NaN,0.378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,-0.1,-0.6,NaN,NaN,NaN,NaN,NaN,NaN,21.0,82.0,0.256,NaN,NaN,NaN,21.0,82.0,0.256,0.256,17.0,31.0,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0,0


In [18]:
url = 'https://www.basketball-reference.com/awards/mvp.html'
output = str(BeautifulSoup(urlopen(url), 'html.parser').findAll('table', id='mvp_NBA')[0])
mvpframe = pd.read_html(output)[0]

In [19]:
# This function is used to fix the BasketballReference tables and convert them into more readable and usable DataFrames
def fix_frame(frame):
    fixed = {   'Season': frame['Unnamed: 0_level_0']['Season'], 
                    'Player': frame['Unnamed: 2_level_0']['Player'],
                    'Age': frame['Unnamed: 4_level_0']['Age'],
                    'Team': frame['Unnamed: 5_level_0']['Tm'],
                    'Games Played': frame['Unnamed: 6_level_0']['G'],
                    'MPG': frame['Per Game']['MP'],
                    'PPG': frame['Per Game']['PTS'],
                    'RPG': frame['Per Game']['TRB'],
                    'APG': frame['Per Game']['AST'],
                    'SPG': frame['Per Game']['STL'],
                    'BPG': frame['Per Game']['BLK'],
                    'FG%': frame['Shooting']['FG%'],
                    '3P%': frame['Shooting']['3P%'],
                    'FT%': frame['Shooting']['FT%'],
                    'WS': frame['Advanced']['WS'],
                    'WS/48': frame['Advanced']['WS']}
    res = pd.DataFrame(data=fixed)
    return res

In [20]:
# Showing the fixed DataFrame
mvpframe = fix_frame(mvpframe)
mvpframe.head()


,Season,Player,Age,Team,Games Played,MPG,PPG,RPG,APG,SPG,BPG,FG%,3P%,FT%,WS,WS/48
0,2021-22,Nikola Jokić,26,DEN,74,33.5,27.1,13.8,7.9,1.5,0.9,0.583,0.337,0.810,15.2,15.2
1,2020-21,Nikola Jokić,25,DEN,72,34.6,26.4,10.8,8.3,1.3,0.7,0.566,0.388,0.868,15.6,15.6
2,2019-20,Giannis Antetokounmpo,25,MIL,63,30.4,29.5,13.6,5.6,1.0,1.0,0.553,0.304,0.633,11.1,11.1
3,2018-19,Giannis Antetokounmpo,24,MIL,72,32.8,27.7,12.5,5.9,1.3,1.5,0.578,0.256,0.729,14.4,14.4
4,2017-18,James Harden,28,HOU,72,35.4,30.4,5.4,8.8,1.8,0.7,0.449,0.367,0.858,15.4,15.4


In [22]:
winloss = pd.read_excel('datafiles/Historical NBA Performance.xlsx')
winloss = winloss.drop(columns=['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'])
winloss['Wins'] = np.zeros(len(winloss), dtype=int)
winloss['Losses'] = np.zeros(len(winloss), dtype=int)
for index, row in winloss.iterrows():
    wl = row['Record'].split('-')
    winloss.loc[index, 'Wins'] = wl[0]
    winloss.loc[index, 'Losses'] = wl[1]
winloss = winloss.drop(columns=['Record'])
winloss

,Year,Team,Winning Percentage,Wins,Losses
0,2016-17,Celtics,0.625,25,15
1,2015-16,Celtics,0.585,48,34
2,2014-15,Celtics,0.488,40,42
3,2013-14,Celtics,0.305,25,57
4,2012-13,Celtics,0.506,41,40
...,...,...,...,...,...
1412,1965-66,Bullets,0.475,38,42
1413,1964-65,Bullets,0.463,37,43
1414,1963-64,Bullets,0.388,31,49
1415,1962-63,Zephyrs,0.313,25,55


In [23]:
teams = pd.read_json('datafiles/teams.json')
teams = teams.drop(columns=['teamId'])
teams

,abbreviation,teamName,simpleName,location
0,ATL,Atlanta Hawks,Hawks,Atlanta
1,BOS,Boston Celtics,Celtics,Boston
2,BKN,Brooklyn Nets,Nets,Brooklyn
3,CHA,Charlotte Hornets,Hornets,Charlotte
4,CHI,Chicago Bulls,Bulls,Chicago
5,CLE,Cleveland Cavaliers,Cavaliers,Cleveland
6,DAL,Dallas Mavericks,Mavericks,Dallas
7,DEN,Denver Nuggets,Nuggets,Denver
8,DET,Detroit Pistons,Pistons,Detroit
9,GSW,Golden State Warriors,Warriors,Golden State


In [25]:
winloss.loc[(winloss['Year'] == '2014-15') & (winloss['Team'] == 'Spurs' )]

,Year,Team,Winning Percentage,Wins,Losses
1257,2014-15,Spurs,0.671,55,27
